In [1]:
import numpy as np
from keras.preprocessing import image
from keras.applications import inception_v3
from keras.models import Model
from keras.layers import Dense, Flatten, Input
from keras.optimizers import SGD, RMSprop, Adam
import vgg16 as jeremy_vgg16
from matplotlib import pyplot as plt

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
# For showing the plots in this webpage
%matplotlib inline

In [3]:
BATCH_SIZE = 64
DATA_PATH = "../data/compcars/data/image/"
batches = jeremy_vgg16.Vgg16().get_batches(DATA_PATH+'train',
                                 gen=image.ImageDataGenerator(preprocessing_function=jeremy_vgg16.vgg_preprocess),
                                 batch_size=BATCH_SIZE)
val_batches = jeremy_vgg16.Vgg16().get_batches(DATA_PATH+'valid',
                                     gen=image.ImageDataGenerator(preprocessing_function=jeremy_vgg16.vgg_preprocess),
                                     batch_size=BATCH_SIZE)

Found 14673 images belonging to 1343 classes.
Found 1343 images belonging to 1343 classes.


In [4]:
input_layer = Input(shape=(3, 224, 224),
              name='image_input')
base_model = inception_v3.InceptionV3(weights='imagenet', include_top=False)
x = base_model(input_layer)
x = Flatten(name='flatten')(x)
x = Dense(4096, activation='softmax', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
predictions = Dense(batches.nb_class, activation='softmax', name='predictions')(x)

In [5]:
# this is the model we will train
keras_vgg = Model(input=input_layer, output=predictions)

# freeze all convolutional Vgg16 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model
keras_vgg.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
# Plot the results of the training
def plot_results(history):
    # Summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # Summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
history = keras_vgg.fit_generator(batches,
                        validation_data=val_batches,
                        samples_per_epoch=batches.nb_sample,
                        nb_val_samples=val_batches.nb_sample,
                        nb_epoch=10)
plot_results(history)
keras_vgg.save_weights('inception_model_10.h5')

KeyboardInterrupt: 

In [ ]:
history = keras_vgg.fit_generator(batches,
                        validation_data=val_batches,
                        samples_per_epoch=batches.nb_sample,
                        nb_val_samples=val_batches.nb_sample,
                        nb_epoch=20)
plot_results(history)
keras_vgg.save_weights('inception_model_30.h5')

In [ ]:
history = keras_vgg.fit_generator(batches,
                        validation_data=val_batches,
                        samples_per_epoch=batches.nb_sample,
                        nb_val_samples=val_batches.nb_sample,
                        nb_epoch=30)
plot_results(history)
keras_vgg.save_weights('inception_model_60.h5')

In [ ]:
history = keras_vgg.fit_generator(batches,
                        validation_data=val_batches,
                        samples_per_epoch=batches.nb_sample,
                        nb_val_samples=val_batches.nb_sample,
                        nb_epoch=40)
plot_results(history)
keras_vgg.save_weights('inception_model_100.h5')

In [ ]:
history = keras_vgg.fit_generator(batches,
                        validation_data=val_batches,
                        samples_per_epoch=batches.nb_sample,
                        nb_val_samples=val_batches.nb_sample,
                        nb_epoch=50)
plot_results(history)
keras_vgg.save_weights('inception_model_150.h5')

In [ ]:
keras_vgg.load_weights('test_model.h5')
print "model loaded"

In [ ]:
test_batches = jeremy_vgg16.Vgg16().get_batches(DATA_PATH+'test',
                                 gen=image.ImageDataGenerator(preprocessing_function=jeremy_vgg16.vgg_preprocess),
                                 batch_size=BATCH_SIZE)

In [ ]:
probs = keras_vgg.predict_generator(test_batches, test_batches.nb_sample)
print probs